In [24]:
import requests
import json
from pathlib import Path
import pandas as pd 

In [2]:
api_url = "https://datasets-server.huggingface.co/rows?dataset=Ammok/apple_stock_price_from_1980-2021&config=default&split=train&offset=0&length=100"

In [3]:
response = requests.get(api_url)

In [4]:
data = response.json()

In [33]:
df = pd.json_normalize(data['rows'])

[{'row_idx': 0,
  'row': {'Date': '1980-12-12',
   'Open': 0.128348,
   'High': 0.128906,
   'Low': 0.128348,
   'Close': 0.128348,
   'Adj Close': 0.100178,
   'Volume': 469033600},
  'truncated_cells': []},
 {'row_idx': 1,
  'row': {'Date': '1980-12-15',
   'Open': 0.12221,
   'High': 0.12221,
   'Low': 0.121652,
   'Close': 0.121652,
   'Adj Close': 0.094952,
   'Volume': 175884800},
  'truncated_cells': []},
 {'row_idx': 2,
  'row': {'Date': '1980-12-16',
   'Open': 0.113281,
   'High': 0.113281,
   'Low': 0.112723,
   'Close': 0.112723,
   'Adj Close': 0.087983,
   'Volume': 105728000},
  'truncated_cells': []},
 {'row_idx': 3,
  'row': {'Date': '1980-12-17',
   'Open': 0.115513,
   'High': 0.116071,
   'Low': 0.115513,
   'Close': 0.115513,
   'Adj Close': 0.09016,
   'Volume': 86441600},
  'truncated_cells': []},
 {'row_idx': 4,
  'row': {'Date': '1980-12-18',
   'Open': 0.118862,
   'High': 0.11942,
   'Low': 0.118862,
   'Close': 0.118862,
   'Adj Close': 0.092774,
   'Volume'

In [38]:
df=df.drop(['row_idx', 'truncated_cells'], axis=1)

,row.Date,row.Open,row.High,row.Low,row.Close,row.Adj Close,row.Volume
0,1980-12-12,0.128348,0.128906,0.128348,0.128348,0.100178,469033600
1,1980-12-15,0.122210,0.122210,0.121652,0.121652,0.094952,175884800
2,1980-12-16,0.113281,0.113281,0.112723,0.112723,0.087983,105728000
3,1980-12-17,0.115513,0.116071,0.115513,0.115513,0.090160,86441600
4,1980-12-18,0.118862,0.119420,0.118862,0.118862,0.092774,73449600
...,...,...,...,...,...,...,...
95,1981-04-30,0.126674,0.127790,0.126674,0.126674,0.098872,12611200
96,1981-05-01,0.126674,0.127790,0.126674,0.126674,0.098872,16553600
97,1981-05-04,0.126674,0.126674,0.126116,0.126116,0.098436,14448000
98,1981-05-05,0.126116,0.126116,0.125558,0.125558,0.098001,17539200


In [22]:
total_line = data['num_rows_total']

In [23]:
q = total_line // 100
r = total_line % 100
n_iter = q if r==0 else q + 1
for i in range(1,n_iter) :  # commence à 1 car on aura déjà fait au moins un request pour obtenir le total_line même s'il est nul
    offset = i*100
    api_url = f'"https://datasets-server.huggingface.co/rows?dataset=Ammok/apple_stock_price_from_1980-2021&config=default&split=train&offset={offset}&length=100"'
    print(api_url)


"https://datasets-server.huggingface.co/rows?dataset=Ammok/apple_stock_price_from_1980-2021&config=default&split=train&offset=0&length=100"
"https://datasets-server.huggingface.co/rows?dataset=Ammok/apple_stock_price_from_1980-2021&config=default&split=train&offset=100&length=100"
"https://datasets-server.huggingface.co/rows?dataset=Ammok/apple_stock_price_from_1980-2021&config=default&split=train&offset=200&length=100"
"https://datasets-server.huggingface.co/rows?dataset=Ammok/apple_stock_price_from_1980-2021&config=default&split=train&offset=300&length=100"
"https://datasets-server.huggingface.co/rows?dataset=Ammok/apple_stock_price_from_1980-2021&config=default&split=train&offset=400&length=100"
"https://datasets-server.huggingface.co/rows?dataset=Ammok/apple_stock_price_from_1980-2021&config=default&split=train&offset=500&length=100"
"https://datasets-server.huggingface.co/rows?dataset=Ammok/apple_stock_price_from_1980-2021&config=default&split=train&offset=600&length=100"
"https:/

In [ ]:
def load_with_for(api_url, output_path) :
    # Preparing loop's parameters
    response = requests.get(api_url)
    data = response.json()
    total_line = data['num_rows_total'] # we could request directly num_rows with the hugg_face params size but with this way we economize the first loading
    df = pd.json_normalize(data['rows'])  # initialiser la transformation des data en csv
    q = total_line // 100
    r = total_line % 100
    n_iter = q if r==0 else q + 1

    # loading data sequentially
    for i in range(1,n_iter) :  # commence à 1 car on aura déjà fait au moins un request pour obtenir le total_line même s'il est nul
        offset = i*100
        api_url = f'"https://datasets-server.huggingface.co/rows?dataset=Ammok/apple_stock_price_from_1980-2021&config=default&split=train&offset={offset}&length=100"'
        response = requests.get(api_url)
        data = response.json()
        df = pd.concat({df,pd.json_normalize(data['rows'])}, axis = 1, ignore_index = True)

    # sortie
    data_csv = df.to_csv(output_path, index=False)  

### old : Exporting data to json file

In [5]:
output_dir = Path("%pwd").resolve().parent

In [6]:
output_path = output_dir / "hugging_face_data.json"

In [7]:
with open(output_path, "w") as f:  # with garantit la fermeture auto du fichier même en cas d'erreur
        json.dump(data, f, indent=2)